In [ ]:
# from transformers import PegasusTokenizer, BigBirdPegasusForConditionalGeneration, BigBirdPegasusConfig
# import pandas as pd

# df = pd.read_json('foxnews_news_cleaned.json')
# model = BigBirdPegasusForConditionalGeneration.from_pretrained('google/bigbird-pegasus-large-arxiv')
# tokenizer = PegasusTokenizer.from_pretrained('google/bigbird-pegasus-large-arxiv')
# ARTICLE_TO_SUMMARIZE = df.body.loc[0] #"My friends are cool but they eat too many carbs."
# inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=4096, return_tensors='pt', truncation=True)
# # Generate Summary
# summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=50, early_stopping=True)
# print(ARTICLE_TO_SUMMARIZE)
# print([tokenizer.decode(g, skip_special_tokens=False, clean_up_tokenization_spaces=False) for g in summary_ids])

In [1]:
import torch
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
#from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline

In [ ]:
#!pip3 uninstall transformers -y
#!pip3 install git+https://github.com/huggingface/transformers
#!pip3 install SentencePiece

In [ ]:
df = pd.read_json('foxnews_news_cleaned.json')
print(df.sentiment.value_counts(normalize=True))
summarizer = pipeline('summarization', model="google/bigbird-pegasus-large-arxiv")
abstract = summarizer(df.body.loc[0], truncation='longest_first')


In [ ]:
abstract

In [ ]:
df = pd.read_json('foxnews_news_cleaned.json')
model_name = 'google/pegasus-xsum'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

# Generate in small batches else running out of memory on GTX 1080Ti :(
summary = []
chunkSize = 5
with torch.no_grad():
    for x in tqdm(range(0,df.shape[0], chunkSize), ncols=100):
        textChunk = df.body.loc[x:x+chunkSize-1]
        batch = tokenizer([f'\"{t}\"' for t in textChunk], truncation=True, padding='longest', return_tensors="pt").to(device)
        translated = model.generate(**batch)
        summary.extend(tokenizer.batch_decode(translated, skip_special_tokens=True))
df['summary'] = summary

In [ ]:
# df = pd.read_json('foxnews_news_cleaned.json')
# model_name = 'google/pegasus-xsum'
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

# tokenizer = PegasusTokenizer.from_pretrained(model_name)
# model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
# batch = tokenizer([df.body.loc[20]], truncation=True, padding='longest', return_tensors="pt").to(device)
# translated = model.generate(**batch)
# tokenizer.batch_decode(translated, skip_special_tokens=True)

In [ ]:
df.summary.value_counts().nlargest(2)

In [ ]:
# Save the cleaned data
parsed = json.loads(df.to_json(orient='records', force_ascii=False, indent=4))
with open('foxnews_news_cleaned_summary.json', 'w', encoding='utf-8') as f:
    json.dump(parsed, f, indent=4)

In [ ]:
df = pd.read_json('foxnews_news_cleaned_summary.json')
df

In [ ]:
a = df[df.summary == 'All images are copyrighted.']
print(a)
a.loc[20].body

In [ ]:
# Split to training/test set
df = pd.read_json('foxnews_news_cleaned_summary.json')
df = df.loc[df['sentiment'] != 'NA', ['summary', 'sentiment']]
label = {'neutral':0,'positive':1,'negative':2}
df['labels'] = df['sentiment'].map(label)

# Split the train/test set
train, test = train_test_split(df[['summary', 'labels']], test_size=0.2, random_state=12, shuffle=True)
train.columns = ['text','labels']
test.columns = ['text','labels']
train.to_csv('data/train.csv', sep=',', index=False)
test.to_csv('data/test.csv', sep=',', index=False)